In [1]:
import numpy as np
import pandas as pd
import requests
import matplotlib.pyplot as plt
import gmaps
import json
import io
import csv
from pandas.io.json import json_normalize
import warnings
import matplotlib.cbook

# Bike Data & gmaps api key
from config import api_key
gmaps.configure(api_key=api_key)

In [2]:
# Inserting error removal, the below .loc line provides a "warning" that is
# a false positive, the data is still correct and what we are looking for
# See http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
# and https://stackoverflow.com/questions/20625582/how-to-deal-with-settingwithcopywarning-in-pandas
# For further information
pd.options.mode.chained_assignment = None  # default='warn'

In [3]:
# Getting data from API, turning into json file
url = "https://municipal.systems/v1/places/az/dataTypes/shared-vehicle/data?key=f867b00f-7408-4c88-bd4d-ad4ef582b20c"
response2 = requests.get(url).json()

# Creating a .txt file to be read into a dataframe
with open('data_files/bike_data.txt', 'w') as outfile2:
  json.dump(response2, outfile2)
with open('data_files/bike_data.txt') as bike_file:
  dict_bike = json.load(bike_file)

# Turning json file into a readable dataframe
bike_data_df = pd.DataFrame.from_dict(json_normalize(dict_bike['results']), orient='columns')

# Formatting column names
bike_data_df.columns = bike_data_df.columns.str.strip().str.lower().str.replace('data.','')

# Dropping unnecessary columns
bike_data_df = bike_data_df[['name','geometry.coordinates','operators',]]

# Removing brackets from operators values, then dropping JUMP LA rows
bike_data_df.loc[:,'operators'] = bike_data_df['operators'].str.get(0)
bike_data_df = bike_data_df[bike_data_df.operators != 'JUMP LA']

# Removing brackets from geometry.coordinates, splitting into lat/lon columns
bike_data_df['coordinates'] = bike_data_df['geometry.coordinates'].astype(str).str.replace('\[|\]','')

# Creating lat/lon lists
lat = []
lon = []

for row in bike_data_df['coordinates']:
   lat.append(row.split(',')[1])
   lon.append(row.split(',')[0])

# appending lat and lon data to new columns in dataframe
bike_data_df['latitude'] = lat
bike_data_df['longitude'] = lon

# # Dropping unnecessary columns
bike_data_df = bike_data_df[['name','operators','latitude','longitude']]
bike_data_df.head()

,name,operators,latitude,longitude
0,T759,Grid Bike Share,33.41814,-111.938395
1,GRID 2220,Grid Bike Share,33.46572,-112.089135
2,GRID 190,Grid Bike Share,33.465703,-112.089142
4,GRID 254,Grid Bike Share,33.447542999999996,-112.10143
5,T990,Grid Bike Share,33.416872,-111.920858


In [4]:
# Creating new column with combined lat/lon for future reverse geocode call
bike_data_df['latlng'] = bike_data_df.apply(lambda row: '{},{}'.format(row['latitude'], row['longitude']), axis=1)
bike_data_df['latlng'] = bike_data_df['latlng'].str.strip()
bike_data_df.head()

,name,operators,latitude,longitude,latlng
0,T759,Grid Bike Share,33.41814,-111.938395,"33.41814,-111.938395"
1,GRID 2220,Grid Bike Share,33.46572,-112.089135,"33.46572,-112.089135"
2,GRID 190,Grid Bike Share,33.465703,-112.089142,"33.465703,-112.089142"
4,GRID 254,Grid Bike Share,33.447542999999996,-112.10143,"33.447542999999996,-112.10143"
5,T990,Grid Bike Share,33.416872,-111.920858,"33.416872,-111.920858"


In [5]:
# Creating new columns to hold geocode data, city, and country
bike_data_df['geocode_data'] =''
bike_data_df['city'] = ''
bike_data_df['country'] = ''

# Creating the reverse geocode function to gather the geocode data from google using lat/lon
def reverse_geocode(latlng):
  result = {}
  url = 'https://maps.googleapis.com/maps/api/geocode/json?latlng={0}'
  request = url.format(latlng)
  api_url = (request + '&key=' + api_key)
  data = requests.get(api_url).json()
  if len(data['results']) > 0:
      result = data['results'][0]
  return result

# Pulling the geocode data
bike_data_df['geocode_data'] = bike_data_df['latlng'].map(reverse_geocode)
bike_data_df

,name,operators,latitude,longitude,latlng,geocode_data,city,country
0,T759,Grid Bike Share,33.41814,-111.938395,"33.41814,-111.938395",{'address_components': [{'long_name': 'Music B...,,
1,GRID 2220,Grid Bike Share,33.46572,-112.089135,"33.46572,-112.089135","{'address_components': [{'long_name': '1145', ...",,
2,GRID 190,Grid Bike Share,33.465703,-112.089142,"33.465703,-112.089142","{'address_components': [{'long_name': '1145', ...",,
4,GRID 254,Grid Bike Share,33.447542999999996,-112.10143,"33.447542999999996,-112.10143","{'address_components': [{'long_name': '1929', ...",,
5,T990,Grid Bike Share,33.416872,-111.920858,"33.416872,-111.920858","{'address_components': [{'long_name': '1122', ...",,
...,...,...,...,...,...,...,...,...
95,GRID 2039,Grid Bike Share,33.45082,-112.075352,"33.45082,-112.075352",{'address_components': [{'long_name': 'Federal...,,
96,GRID 2202,Grid Bike Share,33.452827,-112.075268,"33.452827,-112.075268","{'address_components': [{'long_name': '330', '...",,
97,T856,Grid Bike Share,33.426443,-111.94134,"33.426443,-111.94134","{'address_components': [{'long_name': '410', '...",,
98,T799,Grid Bike Share,33.428087999999995,-111.939832,"33.428087999999995,-111.939832",{'address_components': [{'long_name': '204222'...,,


In [6]:
# identify municipality and country data in the json that google sent back
def parse_city(geocode_data):
   if (not geocode_data is None) and ('address_components' in geocode_data):
       for component in geocode_data['address_components']:
           if 'locality' in component['types']:
               return component['long_name']
           elif 'postal_town' in component['types']:
               return component['long_name']
           elif 'administrative_area_level_2' in component['types']:
               return component['long_name']
           elif 'administrative_area_level_1' in component['types']:
               return component['long_name']
   return None
def parse_country(geocode_data):
   if (not geocode_data is None) and ('address_components' in geocode_data):
       for component in geocode_data['address_components']:
           if 'country' in component['types']:
               return component['long_name']
   return None

In [7]:
# Using the parse city and parse country functions to gather the city and country name from the geo code data
bike_data_df['city'] = bike_data_df['geocode_data'].map(parse_city)
bike_data_df['country'] = bike_data_df['geocode_data'].map(parse_country)

# Stripping extra spaces from the latitude column
bike_data_df['latitude'] = bike_data_df['latitude'].str.strip()

# Verifying number of bike data points
print(len(bike_data_df))

bike_data_df.head()

97


,name,operators,latitude,longitude,latlng,geocode_data,city,country
0,T759,Grid Bike Share,33.41814,-111.938395,"33.41814,-111.938395",{'address_components': [{'long_name': 'Music B...,Tempe,United States
1,GRID 2220,Grid Bike Share,33.46572,-112.089135,"33.46572,-112.089135","{'address_components': [{'long_name': '1145', ...",Phoenix,United States
2,GRID 190,Grid Bike Share,33.465703,-112.089142,"33.465703,-112.089142","{'address_components': [{'long_name': '1145', ...",Phoenix,United States
4,GRID 254,Grid Bike Share,33.447542999999996,-112.10143,"33.447542999999996,-112.10143","{'address_components': [{'long_name': '1929', ...",Phoenix,United States
5,T990,Grid Bike Share,33.416872,-111.920858,"33.416872,-111.920858","{'address_components': [{'long_name': '1122', ...",Tempe,United States


In [8]:
# Turning the latitude and longitude column values into floats for the heatmap
bike_data_df['latitude'] = bike_data_df['latitude'].astype(float)
bike_data_df['longitude'] = bike_data_df['longitude'].astype(float)

In [9]:
# Creating a list of tuples to use in the heatmap for bike locations. 
lat_lon_maps = list(zip(bike_data_df.latitude, bike_data_df.longitude))

# Creating a ROADMAP image
fig = gmaps.figure(map_type='ROADMAP')

# Creating the heatmap layer, and adding it to the figure
heatmap_layer = gmaps.heatmap_layer(lat_lon_maps, max_intensity=7, point_radius=10)
fig.add_layer(heatmap_layer)

fig

Figure(layout=FigureLayout(height='420px'))